## 데이터 전처리

### 전처리 함수 모음

In [ ]:
"""
hackle_properties 데이터프레임에서 결측치를 제거하는 함수

사용예시 :
from ppc_properties import process_properties
h_prop = process_properties(h_prop)
"""
def process_properties(h_prop):
    h_prop = h_prop.dropna()

    return h_prop

"""
시도, 시군, 그외를 대분류, 중분류, 소분류로 구분했습니다.
대분류와 중분류만 존재하는 곳도 있고, 전체가 있는 곳도 있어서
3분류로 나눈 다음에 각각에 맞게 들어가게했습니다.
그 과정에서 대분류에는 몇몇 예외사항들이 있어 예외처리 포함했습니다.

사용예시 :
from ppc_address import process_school_address
v_acc_sch = process_school_address(v_acc_sch)
"""

import pandas as pd
import numpy as np

def process_school_address(v_acc_sch):
    region_mapping = {
        '전북': '전라북도', '인천': '인천광역시', '서울': '서울특별시',
        '대구': '대구광역시', '경북': '경상북도', '경남': '경상남도',
        '경기': '경기도', '강원': '강원도', '충남': '충청남도',
        '제주': '제주특별자치도',
    }

    middle_region_mapping = {
        '서울특별시': ['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구',
                  '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구',
                  '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구'],

        '경기도': ['수원시', '성남시', '용인시', '안양시', '안산시', '과천시', '광명시', '광주시',
               '군포시', '부천시', '시흥시', '김포시', '안성시', '오산시', '의왕시', '이천시', '평택시',
               '하남시', '화성시', '여주시', '양평군', '고양시', '구리시', '남양주시', '동두천시',
               '양주시', '의정부시', '파주시', '포천시', '연천군', '가평군'],

        '인천광역시': ['강화군', '옹진군', '중구', '동구', '미추홀구', '연수구', '남동구', '부평구', '계양구', '서구'],

        '강원도': ['춘천시', '원주시', '강릉시', '동해시', '태백시', '속초시', '삼척시', '홍천군', '횡성군',
               '영월군', '평창군', '정선군', '철원군', '화천군', '양구군', '인제군', '고성군', '양양군'],

        '충청북도': ['청주시', '충주시', '제천시', '보은군', '옥천군', '영동군', '증평군', '진천군',
                '괴산군', '음성군', '단양군'],

        '충청남도': ['천안시', '공주시', '보령시', '아산시', '서산시', '논산시', '계롱시', '당진시',
                '금산군', '부여군', '서천군', '청양군', '홍성군', '예산군', '태안군', '세종특별자치시'],

        '전라북도': ['전주시', '익산시', '군산시', '정읍시', '김제시', '남원시', '완주군', '고창군',
                '부안군', '임실군', '순창군', '진안군', '무주군', '장수군'],

        '전라남도': ['목포시', '여수시', '순천시', '나주시', '광양시', '담양군', '곡성군', '구례군',
                '고흥군', '보성군', '화순군', '장흥군', '강진군', '해남군', '영암군', '무안군',
                '함평군', '영광군', '장성군', '완도군', '진도군', '신안군'],

        '광주광역시': ['동구', '서구', '남구', '북구', '광산구'],

        '대구광역시': ['중구', '동구', '서구', '남구', '북구', '수성구', '달서구', '달성군', '군위군'],

        '대전광역시': ['동구', '중구', '서구', '유성구', '대덕구'],

        '울산광역시': ['중구', '남구', '동구', '북구', '울주군'],

        '경상북도': ['포항시', '경주시', '김천시', '안동시', '구미시', '영주시', '영천시', '상주시',
                '문경시', '경산시', '의성군', '청송군', '영양군', '영덕군', '청도군', '고령군',
                '성주군', '칠곡군', '예천군', '봉화군', '울진군', '울릉군'],

        '경상남도': ['창원시', '김해시', '진주시', '양산시', '거제시', '통영시', '사천시', '밀양시',
                '함안군', '거창군', '창녕군', '고성군', '하동군', '합천군', '남해군', '함양군',
                '산청군', '의령군'],

        '부산광역시': ['중구', '서구', '동구', '영도구', '부산진구', '동래구', '남구', '북구',
                 '해운대구', '사하구', '금정구', '강서구', '연제구', '수영구', '사상구', '기장군'],

        '제주특별자치도': ['제주시', '서귀포시']
    }

    def split_address(address):
        if address == '-' or pd.isna(address):
            return pd.Series([np.nan, np.nan, np.nan])

        parts = address.strip().split()

        if parts[0] == '대한민국':
            parts = parts[1:]

        if not parts:
            return pd.Series([np.nan, np.nan, np.nan])

        # 대분류 처리
        large_region = parts[0]
        for short, full in region_mapping.items():
            if large_region == short:
                large_region = full

        if large_region not in middle_region_mapping.keys():
            return pd.Series([np.nan, np.nan, np.nan])

        # 중분류와 소분류 처리
        middle_region = np.nan
        small_region = np.nan

        if len(parts) > 1:
            # 중분류 확인
            for region in middle_region_mapping.get(large_region, []):
                if parts[1].startswith(region):
                    middle_region = region
                    remaining_parts = parts[2:]
                    if remaining_parts:
                        small_region = ' '.join(remaining_parts)
                    break

            # 중분류를 찾지 못한 경우
            if pd.isna(middle_region):
                small_region = ' '.join(parts[1:])

        return pd.Series([large_region, middle_region, small_region])

    # 주소 분리 적용
    v_acc_sch[['지역_대분류', '지역_중분류', '지역_소분류']] = v_acc_sch['address'].apply(split_address)

    # 컬럼 순서 정리
    columns_order = ['id', 'address', 'student_count', 'school_type', '지역_대분류', '지역_중분류', '지역_소분류']
    v_acc_sch = v_acc_sch[columns_order]
    v_acc_sch = v_acc_sch.dropna(subset=['지역_대분류', '지역_중분류', '지역_소분류'], how='all')

    return v_acc_sch

"""
출석 데이터프레임의 attendance_date_list를 처리하는 함수

사용예시 :
from ppc_attendance import process_attendance
v_acc_atd = process_attendance(v_acc_atd)
"""

def process_attendance(v_acc_atd):
    # attendance_date_list를 리스트로 변환하고 explode
    v_acc_atd['attendance_date_list'] = v_acc_atd['attendance_date_list'].apply(eval)
    v_acc_atd = v_acc_atd.explode('attendance_date_list')

    v_acc_atd = v_acc_atd.rename(columns={'attendance_date_list': 'attendance_date'})

    v_acc_atd['attendance_date'] = pd.to_datetime(v_acc_atd['attendance_date'])
    v_acc_atd = v_acc_atd.sort_values('attendance_date')

    v_acc_atd = v_acc_atd.reset_index(drop=True)

    # id 컬럼 1부터 재설정
    v_acc_atd['id'] = range(1, len(v_acc_atd) + 1)

    # attendance_date NaT 행 제거
    v_acc_atd = v_acc_atd.dropna(subset=['attendance_date'])

    return v_acc_atd

"""
차단 기록 데이터프레임에서 자기 자신을 차단한 기록을 제거하는 함수

사용예시 :
from ppc_blockrecord import process_blockrecord
v_acc_bcrec = process_blockrecord(v_acc_bcrec)
"""

def process_blockrecord(v_acc_bcrec):
    v_acc_bcrec = v_acc_bcrec[v_acc_bcrec['block_user_id'] != v_acc_bcrec['user_id']]

    v_acc_bcrec = v_acc_bcrec.reset_index(drop=True)

    return v_acc_bcrec

"""
이벤트 영수증 데이터프레임에서 user_id가 1577954인 중복 데이터 중 하나만 남기는 함수

사용예시 :
from ppc_eventreceipts import process_eventreceipts
v_evt_rcp = process_eventreceipts(v_evt_rcp)
"""

def process_eventreceipts(v_evt_rcp):
    # 1577954 user_id를 가진 데이터만 분리
    mask_1577954 = v_evt_rcp['user_id'] == 1577954
    df_1577954 = v_evt_rcp[mask_1577954].drop_duplicates(subset=['user_id'])
    df_others = v_evt_rcp[~mask_1577954]

    v_evt_rcp = pd.concat([df_1577954, df_others])

    v_evt_rcp = v_evt_rcp.reset_index(drop=True)

    return v_evt_rcp

"""
accounts_group 데이터프레임의 grade와 class_num을 처리하는 함수

사용예시 :
from pcc_group import process_group
v_acc_grp = process_group(v_acc_grp)
"""

def process_group(v_acc_grp):
    v_acc_grp = v_acc_grp[v_acc_grp['grade'] <= 3]
    v_acc_grp = v_acc_grp[(v_acc_grp['class_num'] >= 1) & (v_acc_grp['class_num'] < 25)]

    v_acc_grp = v_acc_grp.reset_index(drop=True)

    return v_acc_grp

"""
polls_question 데이터프레임에서 question_text가 'vote'인 데이터를 제거하는 함수

사용예시 :
from ppc_polls_question import process_polls_question
v_pol_q = process_polls_question(v_pol_q)
"""

def process_polls_question(v_pol_q):
    # question_text가 'vote'인 데이터 제거
    v_pol_q = v_pol_q[v_pol_q['question_text'] != 'vote']

    v_pol_q = v_pol_q.reset_index(drop=True)

    return v_pol_q

"""
polls_questionpiece 데이터프레임에서 데이터 불일치(is_voted=1, is_skipped=1)를 제거하는 함수

사용예시 :
from ppc_polls_questionpiece import process_polls_questionpiece
v_pol_qp = process_polls_questionpiece(v_pol_qp)
"""

def process_polls_questionpiece(v_pol_qp):
    # is_voted가 1이면서 is_skipped도 1인 불일치 데이터 제거
    v_pol_qp = v_pol_qp[~((v_pol_qp['is_voted'] == 1) & (v_pol_qp['is_skipped'] == 1))]

    v_pol_qp = v_pol_qp.reset_index(drop=True)

    return v_pol_qp

"""
polls_questionset 데이터프레임의 question_piece_id_list를 처리하는 함수
created_at이 opening_time보다 늦은 데이터 제거

사용예시 :
from ppc_polls_questionset import process_polls_questionset
v_pol_qset = process_polls_questionset(v_pol_qset)
"""

def process_polls_questionset(v_pol_qset):
    # question_piece_id_list를 리스트로 변환하고 explode
    v_pol_qset['question_piece_id_list'] = v_pol_qset['question_piece_id_list'].apply(eval)
    v_pol_qset = v_pol_qset.explode('question_piece_id_list')

    v_pol_qset = v_pol_qset.rename(columns={'question_piece_id_list': 'question_piece_id'})

    v_pol_qset['opening_time'] = pd.to_datetime(v_pol_qset['opening_time'])
    v_pol_qset['created_at'] = pd.to_datetime(v_pol_qset['created_at'])

    # created_at이 opening_time보다 늦은 데이터 제거
    v_pol_qset = v_pol_qset[v_pol_qset['opening_time'] >= v_pol_qset['created_at']]

    v_pol_qset = v_pol_qset.sort_values('opening_time')
    v_pol_qset = v_pol_qset.reset_index(drop=True)

    v_pol_qset['id'] = range(1, len(v_pol_qset) + 1)

    return v_pol_qset

"""
타임라인 신고 데이터프레임에서 자기 자신을 신고한 행을 제거하는 함수

사용예시 :
from ppc_timelinereport import process_timelinereport
v_acc_trep = process_timelinereport(v_acc_trep)
"""

def process_timelinereport(v_acc_trep):
    # block_user_id와 user_id가 다른 행만 필터링
    v_acc_trep = v_acc_trep[v_acc_trep['reported_user_id'] != v_acc_trep['user_id']]

    v_acc_trep = v_acc_trep.reset_index(drop=True)

    return v_acc_trep

"""
유저 데이터프레임에서 gender와 group_id가 결측치인 행을 제거하는 함수

사용예시 :
from ppc_timelinereport import process_timelinereport
v_acc_user = process_user(v_acc_user)
"""

def process_user(v_acc_user):
    v_acc_user = v_acc_user.dropna()

    return v_acc_user

"""
'invite_count' 컬럼을 생성하는 함수

사용예시 :
from ppc_usercontacts import process_usercontacts
v_acc_ucon = process_usercontacts(v_acc_ucon)
"""

def estimate_invite_count(row):
    length = len(row['invite_user_id_list']) - 2
    # 초대 인원 추정
    if length <= 0:
        return 0
    elif length <= 7:
        return 1
    elif length <= 14:
        return 2
    elif length <= 21:
        return 3
    elif length <= 28:
        return 4
    else:
        return (length + 1) // 6

def process_usercontacts(v_acc_ucon):
    v_acc_ucon['invite_count'] = v_acc_ucon.apply(estimate_invite_count, axis=1)

    return v_acc_ucon

"""
사용자 질문 기록 데이터프레임에서 데이터 불일치(has_read=0, opened_times>0) 행과
시간 순서가 맞지 않는(answer_updated_at < created_at) 행을 제거하는 함수

사용예시 :
from ppc_userquestionrecord import process_userquestionrecord
v_acc_uqrec = process_userquestionrecord(v_acc_uqrec)
"""

def process_userquestionrecord(v_acc_uqrec):
    v_acc_uqrec['created_at'] = pd.to_datetime(v_acc_uqrec['created_at'])
    v_acc_uqrec['answer_updated_at'] = pd.to_datetime(v_acc_uqrec['answer_updated_at'])

    # has_read가 0이고 opened_times가 0보다 큰 불일치 데이터 제거
    v_acc_uqrec = v_acc_uqrec[~((v_acc_uqrec['has_read'] == 0) & (v_acc_uqrec['opened_times'] > 0))]

    # answer_updated_at이 created_at보다 빠른 데이터 제거
    v_acc_uqrec = v_acc_uqrec[~(v_acc_uqrec['answer_updated_at'] < v_acc_uqrec['created_at'])]

    v_acc_uqrec = v_acc_uqrec.reset_index(drop=True)

    return v_acc_uqrec

### 전처리 실행 함수

In [ ]:
"""
모든 전처리 함수를 순차적으로 실행하는 함수

사용예시:
from function.run_preprocessing import run_preprocessing
v_acc_sch, v_acc_atd, v_acc_bcrec, v_evt_rcp, v_acc_grp, v_pol_q, v_pol_qp, v_pol_qset, v_acc_trep, v_acc_user, v_acc_uqrec = run_preprocessing(v_acc_sch, v_acc_atd, v_acc_bcrec, v_evt_rcp, v_acc_grp, v_pol_q, v_pol_qp, v_pol_qset, v_acc_trep, v_acc_user, v_acc_uqrec)
"""

def run_preprocessing(v_acc_sch, v_acc_atd, v_acc_bcrec, v_evt_rcp, v_acc_grp,
                      v_pol_q, v_pol_qp, v_pol_qset, v_acc_trep, v_acc_user, v_acc_uqrec, v_acc_ucon, h_prop):
    print("전처리 시작...")

    try:
        v_acc_sch = process_school_address(v_acc_sch)
        print("1. School Address 전처리 완료")

        v_acc_atd = process_attendance(v_acc_atd)
        print("2. Attendance 전처리 완료")

        v_acc_bcrec = process_blockrecord(v_acc_bcrec)
        print("3. Block Record 전처리 완료")

        v_evt_rcp = process_eventreceipts(v_evt_rcp)
        print("4. Event Receipts 전처리 완료")

        v_acc_grp = process_group(v_acc_grp)
        print("5. Group 전처리 완료")

        v_pol_q = process_polls_question(v_pol_q)
        print("6. Polls Question 전처리 완료")

        v_pol_qp = process_polls_questionpiece(v_pol_qp)
        print("7. Polls Question Piece 전처리 완료")

        v_pol_qset = process_polls_questionset(v_pol_qset)
        print("8. Polls Question Set 전처리 완료")

        v_acc_trep = process_timelinereport(v_acc_trep)
        print("9. Timeline Report 전처리 완료")

        v_acc_user = process_user(v_acc_user)
        print("10. User 전처리 완료")

        v_acc_uqrec = process_userquestionrecord(v_acc_uqrec)
        print("11. User Question Record 전처리 완료")

        v_acc_ucon = process_usercontacts(v_acc_ucon)
        print("12. User Contacts 전처리 완료")

        h_prop = process_properties(h_prop)
        print("13. Properties 전처리 완료")

        print("13개 데이터 전처리 완료")

        return v_acc_sch, v_acc_atd, v_acc_bcrec, v_evt_rcp, v_acc_grp, v_pol_q, v_pol_qp, v_pol_qset, v_acc_trep, v_acc_user, v_acc_uqrec, v_acc_ucon, h_prop

    except Exception as e:
        print(f"전처리 중 오류 발생: {str(e)}")

### 최종 실행 코드

In [ ]:
import pandas as pd
import numpy as np
import re
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
font_path = 'C:/Windows/Fonts/malgun.ttf'  # 폰트 경로를 시스템에 맞게 변경
font_prop = fm.FontProperties(fname=font_path)
plt.rc('font', family=font_prop.get_name())
plt.rcParams['axes.unicode_minus'] = False   # 음수 부호가 올바르게 표시되도록 설정

# vote
vote_path = 'D:/Data/Codeit/4. 고급프로젝트/csv/votes'

csv_files = [f for f in os.listdir(vote_path) if f.endswith('.csv')]

for file in csv_files:
    var_name = file.replace('.csv', '')

    globals()[var_name] = pd.read_csv(os.path.join(vote_path, file), encoding='utf-8-sig')

# hackle
hackle_path = 'D:/Data/Codeit/4. 고급프로젝트/csv/hackle'

csv_files = [f for f in os.listdir(hackle_path) if f.endswith('.csv')]

for file in csv_files:
    var_name = file.replace('.csv', '')

    globals()[var_name] = pd.read_csv(os.path.join(hackle_path, file), encoding='utf-8-sig')

# vote table
v_acc_atd = accounts_attendance
v_acc_bcrec = accounts_blockrecord
v_acc_failpay = accounts_failpaymenthistory
v_acc_frdreq = accounts_friendrequest
v_acc_grp = accounts_group
v_acc_nbs = accounts_nearbyschool
v_acc_pay = accounts_paymenthistory
v_acc_point = accounts_pointhistory
v_acc_sch = accounts_school
v_acc_trep = accounts_timelinereport
v_acc_user = accounts_user
v_acc_uqrec = accounts_userquestionrecord
v_acc_uwd = accounts_userwithdraw
v_acc_ucon = accounts_user_contacts
v_evt = events
v_evt_rcp = event_receipts
v_pol_q = polls_question
v_pol_qp = polls_questionpiece
v_pol_qrep = polls_questionreport
v_pol_qset = polls_questionset
v_pol_uc = polls_usercandidate

# hackle table
h_dev_ppt = device_properties
h_evt = hackle_events
h_prop = hackle_properties_preprocessed
h_user_prop = user_properties

v_acc_sch, v_acc_atd, v_acc_bcrec, v_evt_rcp, v_acc_grp, v_pol_q, v_pol_qp, v_pol_qset, v_acc_trep, v_acc_user, v_acc_uqrec, v_acc_ucon, h_prop = run_preprocessing(
    v_acc_sch, v_acc_atd, v_acc_bcrec, v_evt_rcp, v_acc_grp, v_pol_q,
    v_pol_qp, v_pol_qset, v_acc_trep, v_acc_user, v_acc_uqrec, v_acc_ucon, h_prop
)